# Ejercicio 1 - API IMDB
Acceso a la API de IMDB. Recordad instalar las librerías necesarias.

El objetivo del ejercicio es la creación de un archivo csv a partir de búsquedas en la API.
La solución no es universal. Podéis realizar cualquier mejora o cambio de código para obtener otro resultado. En ese caso debéis comentar cuál sería vuestro objetivo.
Podéis consultarme cualquier duda o enviarme vuestras soluciones a mi correo electrónico gmunoz4@us.es

In [1]:
import csv
import json
import string
import requests
import datetime
import re
#import urllib2
import pprint, codecs, unicodedata

Execute the API call for searching. Print the keys of the result.

In [2]:
# see http://www.omdbapi.com
# buscamos informacion sobre todas las peliculas de la saga "Star Wars"
url ="http://www.omdbapi.com/?s=Star Wars"
search_results  = requests.get(url).json()

In [3]:
# carga en fichero .json, los "nombres de peliculas" recuperados relativos a la palabra de busqueda
# asi se puede trabajar offline (sin tener que acceder a internet de forma periodica)
# nombre fichero - "imdb_palabra de busqueda_fecha descarga.json" 
now=datetime.datetime.now()
nameMovie="StarWars"
namfile= "imdb_00_"+"StarWars"+"_%i-%i-%i" % (now.year, now.month, now.day)
filejson00 = namfile + ".json"

with open(filejson00, 'w') as outfilejson:
    json.dump(search_results['Search'], outfilejson)
    #json.dump(search_results, outfilejson)

We iterate over the results. We store only the results with type 'movie'.

In [4]:
# sobre el fichero json generado, elimina las "no movie" 
with open(filejson00, 'r') as outfilejson:
    d=json.load(outfilejson)

for i in range(len(d)):
    if d[i]['Type'] != "movie":
        d.pop(i)    

with open(filejson00, 'w') as outfilejson:
    json.dump(d, outfilejson)

In [10]:
# la descarga via API con el parametro t no proporciona todos los atributos que se quieren recuperar;
# dado que si tenemos los ID de las peliculas haremos una llamada a la API por ID para recuperar lo que se requiere

namfile= "imdb_01_"+"StarWars"+"_%i-%i-%i" % (now.year, now.month, now.day)
filejson01 = namfile + ".json"

with open(filejson00, 'r') as outfilejson:
    d=json.load(outfilejson)

# diccionario vacio para recuperar la información que necesitamos
infAll = {}
cont=1
# bucle que llama a la API para buscar por ID la informacion de cada pelicula
for i in range(len(d)):
        imdbID= d[i]['imdbID'] 
        url ="http://www.omdbapi.com/?i="+imdbID
        search_results  = requests.get(url).json()
        infAll[cont]=search_results
        cont+=1
      
with open(filejson01, 'w') as outfilejson:
    json.dump(infAll, outfilejson)

Define the csv file. The headers will be 'Genre', 'Description', 'Type', 'Title' and 'imdbRating'.

In [6]:
# nombre del fichero de salida .csv
filecsv = namfile+".csv"

# inicializa fichero de salida - carga cabecera  
with open (filecsv,'w') as outfilecsv:
    # cabecera fichero - campos
    fieldnames = "imdbID type genre year title description imdbRating".split()
    writer= csv.writer(outfilecsv)
    writer.writerow(fieldnames)

For each movie, we get its info from the API to store in the csv file.

In [8]:
with open(filejson01, 'r') as outfilejson:
   fil=json.load(outfilejson)

with open (filecsv,'a') as outfilecsv:
        writer= csv.writer(outfilecsv)
        
        for i in fil.keys():
            imdbID = fil[i]['imdbID']
            type = fil[i]['Type']
            genre = fil[i]['Genre']
            year = fil[i]['Year']
            title = fil[i]['Title']
            description = fil[i]['Plot']
            imdbRating = fil[i]['imdbRating']
            
            fieldata = [imdbID, type, genre, year, title, description, imdbRating]
            writer.writerow(fieldata)            

In [9]:
import pandas as pd
df=pd.read_csv(filecsv)
dfsort=df.sort_values('year', ascending=False)
dfsort

,imdbID,type,genre,year,title,description,imdbRating
3,tt3748528,movie,"Action, Adventure, Sci-Fi",2016,Rogue One: A Star Wars Story,The Rebel Alliance makes a risky move to steal...,8.1
6,tt2488496,movie,"Action, Adventure, Fantasy",2015,Star Wars: The Force Awakens,Three decades after the defeat of the Galactic...,8.2
7,tt1185834,movie,"Animation, Action, Adventure",2008,Star Wars: The Clone Wars,Anakin Skywalker and Ahsoka Tano must rescue t...,5.8
2,tt0121766,movie,"Action, Adventure, Fantasy",2005,Star Wars: Episode III - Revenge of the Sith,"During the near end of the clone wars, Darth S...",7.6
0,tt0121765,movie,"Action, Adventure, Fantasy",2002,Star Wars: Episode II - Attack of the Clones,"Ten years after initially meeting, Anakin Skyw...",6.7
8,tt0120915,movie,"Action, Adventure, Fantasy",1999,Star Wars: Episode I - The Phantom Menace,Two Jedi Knights escape a hostile blockade to ...,6.5
5,tt0086190,movie,"Action, Adventure, Fantasy",1983,Star Wars: Episode VI - Return of the Jedi,After rescuing Han Solo from the palace of Jab...,8.4
4,tt0080684,movie,"Action, Adventure, Fantasy",1980,Star Wars: Episode V - The Empire Strikes Back,After the rebels have been brutally overpowere...,8.8
1,tt0076759,movie,"Action, Adventure, Fantasy",1977,Star Wars: Episode IV - A New Hope,Luke Skywalker joins forces with a Jedi Knight...,8.7
